In [3]:
# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load(os.environ.get('EDAM_PATH', '../edamontology/EDAM_dev.owl'), format='xml')
#g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...

NameError: name 'ConjunctiveGraph' is not defined

In [4]:
from rdflib import ConjunctiveGraph, Namespace
import json
import os

In [5]:
sample_triple= """
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<a> rdfs:subClassOf <b> , <d> .
<c> rdfs:subClassOf <d> .
<b> rdfs:subClassOf <c> , <e> .
<d> rdfs:subClassOf <e> .

"""
q = """
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX edam:<http://edamontology.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?entity ?direct ?undirect WHERE {
   
    ?entity rdfs:subClassOf ?direct .
    OPTIONAL {?direct rdfs:subClassOf+ ?undirect.}
    
     #FILTER ([?entity,?direct] = [?entity,?undirect]) 
     FILTER (?direct = ?undirect)
    
}
ORDER BY ?x
"""

In [6]:
g2 = ConjunctiveGraph()
g2.parse(data=sample_triple, format="turtle")
print(len(g2))



6


In [7]:
res = g2.query(q)
print(len(res))
for r in res: 
    print(f"x = {r['x']},y = {r['y']},z = {r['z']}") 

0


In [112]:
res = g2.query(q)
print(len(res))
direct_superclass = []
undirect_superclass = []
for r in res: 
    direct_superclass.append([r['x'],r['y']])
    undirect_superclass.append([r['x'],r['z']])
for d in direct_superclass :
    if d in undirect_superclass:
        print(f"{d[0]} has a redundant super class")
    

9
file:///home/llamothe/work/ci_query/a has a redundant super class
file:///home/llamothe/work/ci_query/b has a redundant super class


In [116]:
res = g2.query(q)
print(len(res))
triplet = []
for r in res: 
    triplet.append([r['x'],r['y'],r['z']])

for t in triplet :
    for t2 in triplet:
        if [t[0],t[1]] == [t2[0],t2[2]]:
            print(f"{t[0]} has a redundant super class : {t[1]} is already superclass of {t2[1]}")
    

9
file:///home/llamothe/work/ci_query/a has a redundant super class : file:///home/llamothe/work/ci_query/d is already superclass of file:///home/llamothe/work/ci_query/b
file:///home/llamothe/work/ci_query/b has a redundant super class : file:///home/llamothe/work/ci_query/e is already superclass of file:///home/llamothe/work/ci_query/c
